# PHYS 3009: TeV Gamma-Ray Data Analysis with GammaPy

This jupyter notebook presents a quick analysis of H.E.S.S. observations of the Crab Nebula. The data set is part of the first public test data release. You can find more information here: https://www.mpi-hd.mpg.de/hfm/HESS/pages/dl3-dr1/

The analysis is performed using gammapy, a community-developed, open-source Python package for gamma-ray astronomy (https://docs.gammapy.org/). More information on the individual data analysis steps can be found in the gammapy tutorials: https://docs.gammapy.org/1.1/tutorials/index.html

## Preparation

### Imports
Let's start with importing the modules that we used in the last notebook.

In [ ]:
import matplotlib.pyplot as plt

from numpy import sqrt

import astropy.units as u

from astropy.coordinates import (
    SkyCoord, 
    Angle,
)


from gammapy.utils.check import check_tutorials_setup

from gammapy.data import (
    DataStore,
    EventList,
)

from gammapy.stats import WStatCountsStatistic

from gammapy.maps import Map

### Data download
The following lines check the setup and download the data.

In [ ]:
check_tutorials_setup()

In [ ]:
#import os
#os.environ['GAMMAPY_DATA'] = './gammapy-data'

In [ ]:
data_store = DataStore.from_dir("$GAMMAPY_DATA/1.1/hess-dl3-dr1")

In [ ]:
data_store.info()

Expected output:

```
Data store:
HDU index table:
BASE_DIR: gammapy-data/1.1/hess-dl3-dr1
Rows: 630
OBS_ID: 20136 -- 47829
HDU_TYPE: ['aeff', 'bkg', 'edisp', 'events', 'gti', 'psf']
HDU_CLASS: ['aeff_2d', 'bkg_3d', 'edisp_2d', 'events', 'gti', 'psf_table']


Observation table:
Observatory name: 'N/A'
Number of observations: 105
```

### Run Selection

We set the source position. frame='icrs' indicates that we are using coordinates in right ascension and declination.

In [ ]:
source_pos = SkyCoord(83.63311446*u.deg, 22.01448714*u.deg, frame='icrs')

In [ ]:
selectradius = 2.5*u.deg

In [ ]:
conesearch = data_store.obs_table.select_sky_circle(source_pos, selectradius)

In [ ]:
runlist = conesearch['OBS_ID'].value

In [ ]:
print(runlist)

In [ ]:
len(runlist)

In [ ]:
observations = data_store.get_observations(runlist)

We create a dictionary where we will store final results which we produce along the way.

In [ ]:
final_results = {}

Let's add some information to our results:

In [ ]:
final_results['run list'] = runlist

## Sky Maps

### Geometries

In the last notebook we binned all events into a two-dimensional map with coordinates Right Ascension and Declination:

In [ ]:
map_crab = Map.create(binsz=0.01*u.deg, 
                      width=(5*u.deg, 5*u.deg), 
                      skydir=source_pos, 
                      frame='icrs')

The axes of our map are stored in a geometry:

In [ ]:
map_crab.geom

We now want to bin the events into RA, Dec and energy. We will need a three-dimensional geometry.

In [ ]:
# minimum and maximum energy for the analysis
Emin =0.1*u.TeV
Emax = 50*u.TeV

# number of energy bins in the map
map_nEbins = 6

# width of the map
map_width = 8*u.deg

# bin size of the map
map_binsz = 0.01*u.deg

# maximumum offset
offset_max = 2.5*u.deg

In [ ]:
from gammapy.maps import MapAxis, WcsGeom

In [ ]:
map_energy_axis = MapAxis.from_energy_bounds(Emin, Emax,
                                             nbin=map_nEbins, 
                                             name="energy")

map_geom = WcsGeom.create(skydir=source_pos,
                          axes=[map_energy_axis],
                          width=map_width,
                          binsz=map_binsz)

In [ ]:
map_geom

In [ ]:
#We will also need an axis for true energy:

energy_axis_true = MapAxis.from_energy_bounds(0.1*u.TeV, 100*u.TeV,
                                              nbin=20,
                                              per_decade=True,
                                              name="energy_true")

### MapDatasetMaker
We will now use gammapy code to do the binning of our event lists. Objects that manipulate data in gammapy are called Makers. We start with a DatasetMaker.

In [ ]:
from gammapy.datasets import MapDataset

from gammapy.makers import MapDatasetMaker

In [ ]:
map_empty = MapDataset.create(map_geom, 
                              name='empty', 
                              energy_axis_true=energy_axis_true
                             )

In [ ]:
map_maker = MapDatasetMaker()

We will test everything on the first run:

In [ ]:
obs = observations[0]

In [ ]:
map_dataset = map_maker.run(map_empty, obs)

In [ ]:
map_dataset.counts.plot_grid()

In [ ]:
#map_dataset.counts_off

### SafeMaskMaker
We want to reject badly reconstructed events. This could be events with a very large distance from the camera centre (outside the camera) or at low energies where the sensitivity of the instruments is too low. We will reject all events outside a radius of 2.5 deg and where the sensitivity drops below 10% of the maximum value. The SafeMaskMaker will do that for us.

In [ ]:
map_dataset.mask_image.plot()

In [ ]:
from gammapy.makers import SafeMaskMaker

In [ ]:
safe_mask_maker = SafeMaskMaker(methods=["offset-max", "aeff-max"], 
                                offset_max=offset_max,
                                aeff_percent=10
                               )

In [ ]:
map_dataset = safe_mask_maker.run(map_dataset, obs)

In [ ]:
map_dataset.counts.plot_grid()

In [ ]:
map_dataset.mask_image.plot()

### Ring Background
Finally we use the RingBackgroundMaker to create a background estimate. First we need an exclusion mask which flags all known and potential sources.

In [ ]:
from regions import CircleSkyRegion

In [ ]:
exclusion_regions = [CircleSkyRegion(center=source_pos, radius=0.4*u.deg),  ## exclude the Crab Nebula
                     CircleSkyRegion(center=SkyCoord(183.604, -8.708,       ## RGB J0521+212, recommended in gammapy tutorial
                                                     unit="deg", 
                                                     frame="galactic"),
                                     radius=0.4*u.deg)
                    ]

In [ ]:
exclusion_mask = map_geom.to_image().region_mask(exclusion_regions, inside = False) 

In [ ]:
exclusion_mask.plot()

Now we create the RingBackgroundMaker. We can choose the inner radius of the ring, which must be larger than the largest exclusion region, and the width of the ring.

In [ ]:
from gammapy.makers import RingBackgroundMaker

In [ ]:
ring_bkg_maker = RingBackgroundMaker(exclusion_mask=exclusion_mask,
                                     r_in=0.4*u.deg,
                                     width=0.2*u.deg)

In [ ]:
map_dataset = ring_bkg_maker.run(map_dataset)

In [ ]:
map_dataset.counts.plot_grid()

In [ ]:
map_dataset.counts_off.plot_grid()

### Combination of all runs
We now loop over all runs and stack the individual data sets.

In [ ]:
from gammapy.datasets import Datasets

In [ ]:
map_datasets = Datasets()

for obs in observations:
    map_dataset = map_maker.run(map_empty.copy(name = str(obs.obs_id)), obs)
    map_dataset = safe_mask_maker.run(map_dataset, obs)
    map_dataset = ring_bkg_maker.run(map_dataset)
    
    map_datasets.append(map_dataset)

In [ ]:
map_stacked = map_datasets.stack_reduce()

In [ ]:
map_stacked.counts.plot_grid()

In [ ]:
map_on = map_stacked.counts.sum_over_axes()

In [ ]:
map_on.plot(add_cbar = True)

In [ ]:
map_on.smooth(0.05*u.deg).plot(add_cbar = True,
                               cmap = 'coolwarm')

#plt.savefig('MapOn.svg')

In [ ]:
map_stacked.counts_off.plot_grid()

In [ ]:
map_stacked.alpha.plot_grid()

In [ ]:
map_off = (map_stacked.counts_off * map_stacked.alpha).sum_over_axes()

In [ ]:
map_off.plot(add_cbar = True)

In [ ]:
map_off.smooth(0.05*u.deg).plot(add_cbar = True,
                                cmap = 'coolwarm')

#plt.savefig('MapOff.svg')

In [ ]:
map_excess = map_on - map_off

In [ ]:
map_excess.plot(add_cbar = True)

In [ ]:
map_excess.smooth(0.05*u.deg).plot(add_cbar = True,
                                   cmap = 'coolwarm')

#plt.savefig('MapExcess.svg')

### Excess and Significance Maps
We could calculate the significance in each bin. There is an easy way to do that. We can use gammapy's ExcessMapEstimator.

In [ ]:
from gammapy.estimators import ExcessMapEstimator

The ExcessMapEstimator uses oversampling. So we need to decide our oversampling radius. For that we should have a look at our point-spread function (PSF).

In [ ]:
map_stacked.psf.plot_containment_radius_vs_energy(fraction=(0.34, 0.68, 0.95, 0.99))

The 68% radius is often a good choice. But the Crab Nebula is so bright that we can go lower than that. Let's take 0.05 deg.

In [ ]:
estimator = ExcessMapEstimator(0.05*u.deg)

fluxmaps = estimator.run(map_stacked)

In [ ]:
fluxmaps.npred_excess.plot(add_cbar = True, cmap = 'coolwarm')

#plt.savefig('FinalMap_Excess.svg')

In [ ]:
fluxmaps.sqrt_ts.plot(add_cbar = True, cmap = 'coolwarm')

#plt.savefig('FinalMap_Significance.svg')

In [ ]:
fluxmaps.flux.plot(add_cbar = True, cmap = 'coolwarm')

#plt.savefig('FinalMap_Flux.svg')

Let's keep the results for later.

In [ ]:
final_results['excess map'] = fluxmaps.npred_excess.copy()
final_results['significance map'] = fluxmaps.sqrt_ts.copy()
final_results['flux map'] = fluxmaps.flux.copy()

#### your playground
Please try different convolution radii. You will see that smaller radii will lead to a more noisy image and larger radii will make the source appear bigger. This does not mean that the source is indeed bigger. You just smear out the emission over a larger area.

In [ ]:
## your code here

### Evaluation of the Sky Map
We need to check that our background estimate is reasonable and we would like to know if there are any other potential sources in the map.

We will make a histogram of the significance distribution. This will require some data manipulation.v We will make use of numpy, which is typically imported as np.

In [ ]:
import numpy as np

In [ ]:
sigmapdata = final_results['significance map'].data

In [ ]:
sigmapdata

Let's remove anything that is not a number.

In [ ]:
np.isfinite(sigmapdata)

In [ ]:
sigmapdata[np.isfinite(sigmapdata)]

In [ ]:
sig_hist = plt.hist(sigmapdata[np.isfinite(sigmapdata)],
                    bins = 70,
                    density = True,
                    color = 'tab:blue'
                   )

plt.yscale('log')

#plt.savefig('SigDist_all.svg')

We clearly see that there are bins with significances up to 30 sigma (this is the Crab Nebula). We also many bins with significances around 0 (this is in the parts of the image with no source, which is our baclground). We can also study the regions containing only background, we need to multiply the significance image with the exclusion mask. 

In [ ]:
sig_off = final_results['significance map'].reduce_over_axes()*exclusion_mask

In [ ]:
sig_off.plot(add_cbar = True,
             cmap = 'coolwarm'
            )

#plt.savefig('SigMap_masked.svg')

In [ ]:
sigmapdata_off = sig_off.data[np.isfinite(sig_off.data)]

In [ ]:
sig_hist_off = plt.hist(sigmapdata_off,
                        bins = sig_hist[1],
                        density = True
                       )

plt.yscale('log')

This should be a Gaussian centred on 0 with a width of 1. Let's test it.

In [ ]:
from scipy.stats import norm

In [ ]:
mu, std = norm.fit(sigmapdata_off)

print(f'Fit results: mu = {mu:.2f}, std = {std:.2f}')

In [ ]:
x = np.linspace(-5, 5, 50)
p = norm.pdf(x, mu, std)

In [ ]:
plt.stairs(sig_hist_off[0], sig_hist_off[1],
           fill = True,
           color = 'tab:red'
          )

plt.plot(x, p, lw=2, color="black")

plt.yscale('log')

#plt.savefig('SigDist_off.svg')

In [ ]:
plt.stairs(sig_hist[0], sig_hist[1],
           fill = True,
           color = 'tab:blue',
           label='all bins'
          )

plt.stairs(sig_hist_off[0], sig_hist_off[1],
           fill = True,
           color = 'tab:red',
           label = 'off bins',
           alpha = 0.5
          )

plt.plot(x, p, lw=2, color="black", 
         label = f'mu = {mu:.2f}, std = {std:.2f}')

plt.yscale('log')

plt.xlabel('significance')

plt.legend()

#plt.savefig('SigDist_final.svg')

## Energy Spectrum
We now want to study the energy distribution of the emission from our object.

Remember that we plotted the energy distribution of all the events in one observation run:

In [ ]:
obs.events.plot_energy()

#plt.savefig('Counts_vs_Energy.svg')

These event counts were recorded within a certain observation time. We need to divide the counts by the time to get a rate (counts per second).

In [ ]:
obs.obs_info['LIVETIME']  ## observation time in seconds

But we want to measure a flux. So we still need to divide by the effective area of the instrument. The effective area depends on the energy, but also on the conditions of the observations (like the distance of the source from the camera centre).

In [ ]:
obs.aeff.plot_energy_dependence()

plt.xlim(0.4)

#plt.savefig('Aeff.svg')

In [ ]:
obs.edisp.peek()

In [ ]:
obs.edisp.to_edisp_kernel(0.7*u.deg).plot_matrix()

#plt.savefig('EDisp.svg')

In [ ]:
obs.edisp.plot_migration(offset = 0*u.deg, energy_true = [1,10]*u.TeV)

plt.legend(loc='upper right')

#plt.savefig('Eresolution.svg')

### SpectrumDatasetMaker
We will now use gammapy code to do the binning of our event lists. This is very similar to the generation of the MapDataset.

In [ ]:
from gammapy.datasets import SpectrumDataset

from gammapy.makers import SpectrumDatasetMaker

We want to use only the events from a small region around the source, the on region. We start with a region at the source position and with a radius of 0.1 degrees. We can change that later if needed.

In [ ]:
on_region_radius = Angle(0.1*u.deg)

on_region = CircleSkyRegion(center=source_pos, radius=on_region_radius)

We can check on the sky map if the region really encompasses all of the emission. If not, we we need to increase the size. We will use a zoomed version of the significance map.

In [ ]:
skymap = final_results['significance map'].cutout(source_pos, 1*u.deg)

In [ ]:
skymap.plot(add_cbar = True, cmap = 'coolwarm')

on_region.to_pixel(skymap.geom.wcs).plot(color = 'white')

In [ ]:
Emin, Emax

In [ ]:
spectrum_nEbins = 20

spectrum_energy_axis = MapAxis.from_energy_bounds(Emin, Emax,
                                                  nbin=spectrum_nEbins,
                                                  name="energy")

In [ ]:
from gammapy.maps import RegionGeom

In [ ]:
spectrum_geom = RegionGeom.create(region=on_region, 
                                  axes=[spectrum_energy_axis]
                                 )

In [ ]:
spectrum_empty =  SpectrumDataset.create(geom=spectrum_geom, 
                                         energy_axis_true=energy_axis_true
                                        )

In [ ]:
from gammapy.makers import SpectrumDatasetMaker

In [ ]:
spectrum_maker = SpectrumDatasetMaker()

In [ ]:
spectrum_dataset = spectrum_maker.run(spectrum_empty, obs)

In [ ]:
spectrum_dataset.counts.plot()

In [ ]:
#spectrum_dataset.counts_off.plot()

In [ ]:
spectrum_dataset.mask.plot()

### SafeMaskMaker
As before, we need to select the good energy and offset range. We will use the same SafeMaskMaker as before.

In [ ]:
spectrum_dataset = safe_mask_maker.run(spectrum_dataset, obs)

In [ ]:
spectrum_dataset.mask.plot()

In [ ]:
spectrum_dataset.energy_range_total

In this run we will use only events with energies of more than 645 GeV.

### Reflected Background
We will use the reflected background to estimate the off source counts. We will need an exclusion mask, we will use the same as before.

In [ ]:
from gammapy.makers import ReflectedRegionsBackgroundMaker

In [ ]:
reflected_bkg_maker = ReflectedRegionsBackgroundMaker(exclusion_mask=exclusion_mask)

In [ ]:
spectrum_dataset = reflected_bkg_maker.run(spectrum_dataset, obs)

In [ ]:
spectrum_dataset.counts_off.plot()

We can check on the sky map the location of our off-source regions.

In [ ]:
from gammapy.visualization import plot_spectrum_datasets_off_regions

In [ ]:
skymap = final_results['significance map']

ax = skymap.plot()

on_region.to_pixel(skymap.geom.wcs).plot(color = 'white')

plot_spectrum_datasets_off_regions(ax=ax, datasets=[spectrum_dataset])

### Combination of all runs
We now loop over all runs and stack the individual data sets.

In [ ]:
%%time

spectrum_datasets = Datasets()

for obs in observations:
    spectrum_dataset = spectrum_maker.run(spectrum_empty.copy(name = str(obs.obs_id)), obs)
    spectrum_dataset = safe_mask_maker.run(spectrum_dataset, obs)
    spectrum_dataset = reflected_bkg_maker.run(spectrum_dataset, obs)
    
    spectrum_datasets.append(spectrum_dataset)

In [ ]:
spectrum_stacked = spectrum_datasets.stack_reduce()

In [ ]:
ax = skymap.plot(cmap = 'coolwarm', add_cbar = True)

on_region.to_pixel(skymap.geom.wcs).plot(color = 'white')

plot_spectrum_datasets_off_regions(ax=ax, datasets=spectrum_datasets)

#plt.savefig('ReflectedRegions.svg')

### Excess and Significance
We can now use the SpectrumDatasets to study the excess and the significance of the source.

In [ ]:
info_table = spectrum_datasets.info_table()

In [ ]:
info_table.colnames

In [ ]:
info_table['name', 'counts', 'counts_off', 'alpha', 'excess', 'sqrt_ts']

In [ ]:
#info_table['name', 'counts', 'counts_off', 'alpha', 'excess', 'sqrt_ts'].write('info_table.csv')

In [ ]:
sum_table = spectrum_datasets.info_table(cumulative=True)

In [ ]:
sum_table['name', 'counts', 'counts_off', 'alpha', 'excess', 'sqrt_ts']

In [ ]:
#sum_table['name', 'counts', 'counts_off', 'alpha', 'excess', 'sqrt_ts'].write('sum_table.csv')

The last line gives us the on and off counts, as well as excess and significance.

In [ ]:
last_line = sum_table[-1]

In [ ]:
last_line['name', 'counts', 'counts_off', 'alpha', 'excess', 'sqrt_ts']

In [ ]:
last_line['counts'] - last_line['alpha'] * last_line['counts_off']

In [ ]:
stat = WStatCountsStatistic(n_on=last_line['counts'], 
                            n_off=last_line['counts_off'], 
                            alpha=last_line['alpha']
                           )

print('excess: {} \nsignificance: {}'.format(stat.n_sig,stat.sqrt_ts))

Let's keep these results for later.

In [ ]:
final_results['excess'] = sum_table[-1]['excess']
final_results['significance'] = sum_table[-1]['sqrt_ts']

The excess should increase roughly linearly with time:

In [ ]:
plt.errorbar(sum_table['livetime'].to(u.h),
             sum_table['excess'],
             np.sqrt(sum_table['excess']),
             marker='o',
             ls='none'
            )

plt.xlabel('Livetime [h]')
plt.ylabel('Excess')

#plt.savefig('ExcessVsT.svg')

In [ ]:
plt.errorbar(sum_table['livetime'].to(u.h),
             sum_table['sqrt_ts'],
             1,
             marker='o',
             ls='none'
            )

plt.xlabel('Livetime [h]')
plt.ylabel('Significance')

#plt.savefig('SigVsT.svg')

### Excess and Flux

We can subtract the off-source counts from the on-source counts to get the excess:

In [ ]:
spectrum_stacked.plot_counts()

#plt.savefig('Spectrum_OnOffCounts.svg')

As usual, we can use (on - alpha*off) to calculate the excess:

In [ ]:
excess = spectrum_stacked.counts.data - spectrum_stacked.counts_off.data * spectrum_stacked.alpha.data

In [ ]:
excess = excess.reshape(excess.shape[0])

In [ ]:
spectrum_stacked.geoms['geom'].axes['energy'].center

In [ ]:
plt.plot(spectrum_stacked.geoms['geom'].axes['energy'].center,
         excess,
         marker = 'o',
         ls = 'none'
        )

plt.xscale('log')

In [ ]:
spectrum_stacked.plot_excess()

#plt.savefig('Spectrum_ExcessCounts.svg')

We could divide the excess by the exposure to get a flux:

In [ ]:
spectrum_stacked.exposure.plot()

#plt.savefig('Spectrum_Exposure.svg')

But the binning is different and the exposure is in true energy. So we need to apply the energy dispersion matrix and to interpolate. We don't need to do that by hand.

In [ ]:
if False :

    flux = spectrum_dataset.excess.quantity / spectrum_dataset.exposure.quantity

    plt.plot(spectrum_stacked.geoms['geom'].axes['energy'].center,
             flux.reshape(20),
             marker = 'o',
             ls = 'none'
            )

    plt.xlabel('E [{}]'.format(spectrum_stacked.geoms['geom'].axes['energy'].center.unit))
    plt.ylabel('flux [{}]'.format(flux.unit))

    plt.xscale('log')
    plt.yscale('log')

    #plt.savefig('SimpleFlux.svg')

### Spectrum Fit
Now we want to describe the energy distribution of the gamma rays with a function. We will make a spectral fit. We will stack all the data of all runs into one data set and proceed with the fit. We could also fit the model to each run individually.

We start with a simple power law. Remember, the power law is
$$
f(E) = A \times \left( \frac{E}{E_0} \right) ^{-\Gamma}.
$$
The amplitude $A$ and the spectral index $\Gamma$ are free parameters in the fit. $E_0$ is the reference energy, which is not fitted. You can freely chose the value of $E_0$, but it is best to keep it within the energy range of the data.

In [ ]:
from gammapy.modeling.models import PowerLawSpectralModel, SkyModel

In [ ]:
spectral_model = PowerLawSpectralModel(index=2, 
                                       amplitude=1e-11 * u.Unit("cm-2 s-1 TeV-1"), 
                                       reference=1 * u.TeV
                                      )

We set the spectral model to a power law with some meaningful start parameters. The spectral model is only a part of a more general model, the SkyModel, which can also contain a spatial and a temporal model.

In [ ]:
model = SkyModel(spectral_model=spectral_model, name="Crab")

Next we set this model as the model of our Dataset. We can also have a look how well our start parameters describe the data already:

In [ ]:
spectrum_dataset.models = model

In [ ]:
spectrum_dataset.plot_excess()

plt.ylim(1e-1)

To do the fit we use a gammapy object called Fit. We can reuse the same object later, we need to create it only once.

In [ ]:
from gammapy.modeling import Fit

In [ ]:
fit = Fit()

Now we run the fit of our Dataset.

In [ ]:
fit.run(spectrum_dataset)

It is very important to check the output. Success must be True and everything has to terminate succesfully. Otherwise our fit result is wrong. If the fit fails, try to run it again. Or change the start parameters and run again. Do not continue with a failed fit!

We can now take a look at our result and the fit parameters:

In [ ]:
ax_spectrum, ax_residuals = spectrum_dataset.plot_fit()

ax_spectrum.set_ylim(0.1, 40)

#plt.savefig('PLfit.svg')

In [ ]:
spectrum_dataset.models.to_parameters_table()

We will make a copy of the model for later use:

In [ ]:
bestmodel_PL = spectrum_dataset.models.copy()

In [ ]:
from gammapy.modeling.models import create_crab_spectral_model

In [ ]:
fig, ax = plt.subplots()

plot_kwargs = {
    "energy_bounds": [0.1, 30] * u.TeV,
    "ax": ax,
}

bestmodel_PL[0].spectral_model.plot(**plot_kwargs, label="this work")
bestmodel_PL[0].spectral_model.plot_error(facecolor="blue", alpha=0.3, **plot_kwargs)

create_crab_spectral_model("hess_pl").plot(
    **plot_kwargs,
    label="Crab reference",
)

ax.legend()

#plt.savefig('PLmodel.svg')

We will also compare this model with a different model. In order to decide which model to be used we will save the test statistics.

In [ ]:
TS_PL = spectrum_dataset.stat_sum()

In [ ]:
TS_PL

Let's do the spectrum again, this time we want to fit a power law with an exponential cut-off at high energies. This function is defined as
$$
f(E) = A \times \left( \frac{E}{E_0} \right)^{-\Gamma} \times \exp \left(-\frac{E}{E_c} \right).
$$
The last term can be written as
$$
\exp \left(-\frac{E}{E_c} \right) = \exp \left(-\lambda E \right)
$$
with $\lambda = 1/E_c$.

Here it is a good idea to use our best fit parameters from the power law as starting parameters.

In [ ]:
from gammapy.modeling.models import ExpCutoffPowerLawSpectralModel

In [ ]:
bestmodel_PL.parameters['index'].quantity

In [ ]:
bestmodel_PL.parameters['amplitude'].quantity

In [ ]:
spectral_model = ExpCutoffPowerLawSpectralModel(index=bestmodel_PL.parameters['index'].quantity, 
                                                amplitude=bestmodel_PL.parameters['amplitude'].quantity, 
                                                reference=1 * u.TeV, 
                                                lambda_ = 1./(10*u.TeV)
                                               )

model = SkyModel(spectral_model=spectral_model, name="Crab")

spectrum_dataset.models = model

In [ ]:
spectrum_dataset.plot_excess()

plt.ylim(1e-1)

In [ ]:
fit.run(spectrum_dataset)

Don't forget to check for success!

In [ ]:
ax_spectrum, ax_residuals = spectrum_dataset.plot_fit()

ax_spectrum.set_ylim(0.1, 40)

#plt.savefig('ExpPLfit.svg')

In [ ]:
spectrum_dataset.models.to_parameters_table()

In [ ]:
bestmodel_expPL = spectrum_dataset.models.copy()

What is the best-fit cut-off energy?

In [ ]:
1/(bestmodel_expPL.parameters['lambda_'].quantity)

In [ ]:
fig, ax = plt.subplots()

plot_kwargs = {
    "energy_bounds": [0.1, 30] * u.TeV,
    "ax": ax,
}

bestmodel_expPL[0].spectral_model.plot(**plot_kwargs, label="this work")
bestmodel_expPL[0].spectral_model.plot_error(facecolor="blue", alpha=0.3, **plot_kwargs)

create_crab_spectral_model("hess_ecpl").plot(
    **plot_kwargs,
    label="Crab reference",
)

ax.legend()

#plt.savefig('ExpPLmodel.svg')

In [ ]:
TS_expPL = spectrum_dataset.stat_sum()

In [ ]:
TS_expPL

### Compare the Models

Which model is better, with or without the cut-off? We will need to compare the test statistics of the fits.

In [ ]:
print(TS_PL, TS_expPL)

Gammapy uses a log-likelihood fit. So we can use the likelihood and Wilk's theorem, provided that we are dealing with nested models. The TS value returned by gammapy is already 2xln(L). So only need to take the difference:

In [ ]:
TS = TS_PL-TS_expPL
print(TS)

In [ ]:
import scipy.stats

In [ ]:
P = scipy.stats.chi2.sf(TS,1)

print('probabilty: ',P)

In [ ]:
print('significant?', P < 2.7e-3)

The model with the cut-off is not significantly better. So we will use the straight power law for our further analysis.

In [ ]:
bestmodel = bestmodel_PL

Let's set this model as the current model of the Dataset:

In [ ]:
spectrum_dataset.models = bestmodel

And we store the fit parameters in the final results.

In [ ]:
final_results['fit parameters'] = bestmodel.parameters.to_table()

final_results['model type'] = bestmodel[0].spectral_model.tag[0]

### Your playground
You can try to fit yet another model. A LogParabolaSpectralModel could work as well. Check the documentation (https://docs.gammapy.org/0.18.2/api/gammapy.modeling.models.LogParabolaSpectralModel.html#gammapy.modeling.models.LogParabolaSpectralModel) for the parameters of this model.
You can compare the log-parabola model with the power law. But you cannot use Wilk's theorem to compare it to the exponential cut-off power law, as they are not nested models.

In [ ]:
## your code here

## Flux Points
In the final step we want to create flux points which can be used for later analysis and astrophysical modelling. These points will depend on our best-fit model. Let's check first that we indeed have our best-fit model.

In [ ]:
spectrum_dataset.models[0].spectral_model.tag[0]

In [ ]:
spectrum_dataset.models.parameters.to_table()

We need only flux points within our energy range:

In [ ]:
spectrum_dataset.energy_range_total

We will use 10 bins.

In [ ]:
energy_edges = np.geomspace(0.65, 50, 11) * u.TeV

The FluxPointsEstimator will do the analysis.

In [ ]:
from gammapy.estimators import FluxPointsEstimator

In [ ]:
fpe = FluxPointsEstimator(energy_edges = energy_edges,
                          selection_optional = ['ul']
                         )

In [ ]:
flux_points = fpe.run(spectrum_dataset)

In [ ]:
flux_points.to_table()

Let's keep that for later.

In [ ]:
final_results['flux points'] = flux_points.to_table()

In [ ]:
flux_points.plot()

#plt.savefig('Flux_dNdE.svg')

Flux points are often plotted in $E^2 \times dN/dE$:

In [ ]:
flux_points.plot(sed_type = 'e2dnde')

#plt.savefig('Flux_E2dNdE.svg')

We can save the points for later use.

In [ ]:
flux_points.write('Crab_spectrum.ecsv',
                  overwrite = True
                 )

If we want to plot the spectral points with our best-fit model then we best use a FluxPointsDataset.

In [ ]:
from gammapy.datasets import FluxPointsDataset

In [ ]:
flux_points_dataset = FluxPointsDataset(data = flux_points,
                                        models = bestmodel
                                       )

In [ ]:
flux_points_dataset.plot_fit()

#plt.savefig('FluxPoints_wModel.svg')

## Your playground
You can make a spectrum with more or less data points.

In [ ]:
## your code here

## Summary

That's all. Let's see what we have and summarise.

In [ ]:
final_results.keys()

In [ ]:
print('We have found {} runs'. format(len(final_results['run list'])))

In [ ]:
print('We have detected an excess of {:4.1f} gamma rays with a statistical significance of {:3.1f} sigma.'.format(final_results['excess'], final_results['significance']))

In [ ]:
print('The spectrum is best described by a {:s}.'.format(final_results['model type']))

In [ ]:
print('The best fit parameters are:\n', final_results['fit parameters'])

In [ ]:
print('The spectral data points are:\n', final_results['flux points'])

In [ ]:
final_results['excess map'].plot(cmap = 'coolwarm', add_cbar = 'true')

In [ ]:
final_results['significance map'].plot(cmap = 'coolwarm', add_cbar = 'true')